### Retrieval-Augmented Generation
`Query -> Search a Database -> Relevant Documents -> Send to LLM -> Contextually Relevant Answer` <br/>

Complexity from decisions based on:
- Chunking.
- Databases.
- Preprocessing query.
- Postprocessing results.
- Semantic vs Keywords.
- Hypothetical searches.
- Multi-hop retrieval.
- Agentic retrieval.

#### Multi-Hop Retrieval
`Question -> LM <-> Hybrid Search from DB` <br/>
`Context -> LM <-> DB` <br/>
`Context -> LM -> Answer` <br/>

#### Hybrid HyDE Search
`Question -> HyDE LM -> (Semantic Query -> Embedding Search) + (BM-25 Query -> BM-25 Search) -> Reciprocal Rank Fusion`

### Setup Jokes DB
<a href="https://www.kaggle.com/datasets/abhinavmoudgil95/short-jokes">Dataset link.</a>

In [ ]:
import torch
import numpy as np
from transformers import DistilBertModel, DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def embed_texts(texts):
    encoded_input = tokenizer(texts, padding=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state[:,0,:].numpy()
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    
    return embeddings

In [2]:
import pandas as pd
from tqdm import tqdm

data = pd.read_csv('shortjokes.csv')
jokes = data['Joke'].values
jokes = jokes[:5000]
batch_size = 512
all_embeddings = []
for i in tqdm(range(0, len(jokes), batch_size), desc='Generating embeddings'):
    batch_texts = jokes[i:i+batch_size].tolist()
    batch_embeddings = embed_texts(batch_texts)
    all_embeddings.append(batch_embeddings)

embeddings = np.concatenate(all_embeddings, axis=0)
print(f'Total embeddings: {len(embeddings)}')
np.save('embeddings.npy', embeddings)
with open('jokes.txt', 'w') as f:
    for joke in jokes:
        f.write(joke+'\n')


Generating embeddings: 100%|██████████| 10/10 [01:45<00:00, 10.54s/it]

Total embeddings: 5000


### JokeGenerator Example
`Query -> (Idea LM <-> WebSearch) -> Joke Idea -> (Joke LM <-> Joke DB) -> Joke`